In [3]:
from glob import glob
from os.path import join
import pandas as pd
import numpy as np

In [4]:
common_path = '/Volumes/MMIS-Saraiv/Datasets/Miltiadous Dataset/features'
all_directories = glob(join(common_path, '*'))
all_directories = [directory for directory in all_directories if not directory.endswith('.csv')]

In [5]:
FEATURES_SELECTED = ['Spectral#RelativePower#C3#beta1', 'Spectral#EdgeFrequency#C3#beta3', 'Spectral#RelativePower#C3#gamma', 'Spectral#EdgeFrequency#C4#alpha1', 'Spectral#RelativePower#C4#beta3', 'Spectral#EdgeFrequency#C4#beta3', 'Spectral#EdgeFrequency#C4#gamma', 'Spectral#Flatness#Cz#theta', 'Spectral#PeakFrequency#Cz#theta', 'Spectral#EdgeFrequency#Cz#beta3', 'Spectral#EdgeFrequency#Cz#gamma', 'Spectral#PeakFrequency#Cz#gamma', 'Spectral#RelativePower#F3#beta1', 'Spectral#Diff#F4#delta', 'Spectral#RelativePower#F7#beta3', 'Spectral#EdgeFrequency#F7#beta3', 'Spectral#RelativePower#F7#gamma', 'Spectral#RelativePower#F8#beta1', 'Spectral#EdgeFrequency#F8#beta3', 'Spectral#RelativePower#Fp1#beta1', 'Spectral#EdgeFrequency#Fp1#beta3', 'Spectral#Diff#Fp2#delta', 'Spectral#RelativePower#Fp2#beta1', 'Spectral#RelativePower#Fp2#beta3', 'Spectral#Diff#Fpz#beta2', 'Spectral#Entropy#O1#delta', 'Spectral#RelativePower#O1#beta2', 'Spectral#EdgeFrequency#O1#beta2', 'Spectral#EdgeFrequency#O1#beta3', 'Spectral#RelativePower#O2#delta', 'Spectral#PeakFrequency#O2#alpha1', 'Spectral#RelativePower#O2#beta1', 'Spectral#RelativePower#O2#beta3', 'Spectral#Diff#P3#beta1', 'Spectral#RelativePower#P3#beta3', 'Spectral#RelativePower#Pz#alpha1', 'Spectral#EdgeFrequency#Pz#beta3', 'Spectral#RelativePower#T4#alpha1', 'Spectral#RelativePower#T4#beta3', 'Spectral#RelativePower#T4#gamma', 'Spectral#EdgeFrequency#T5#beta2', 'Hjorth#Complexity#T5', 'Hjorth#Complexity#P4', 'Hjorth#Complexity#F7', 'Hjorth#Complexity#T4', 'Hjorth#Complexity#F8', 'Hjorth#Complexity#T3', 'Hjorth#Mobility#P3', 'PLI#Frontal(L)-Temporal(R)#alpha1', 'PLI#Frontal(L)-Occipital(L)#alpha1', 'PLI#Frontal(R)-Temporal(R)#alpha1', 'PLI#Temporal(R)-Parietal(R)#alpha1', 'PLI#Temporal(R)-Occipital(L)#alpha1', 'PLI#Parietal(R)-Occipital(L)#alpha1', 'PLI#Occipital(L)-Occipital(R)#alpha1', 'PLI#Temporal(R)-Occipital(R)#alpha2', 'PLI#Parietal(R)-Occipital(L)#alpha2', 'COH#Frontal(L)-Frontal(R)#theta', 'COH#Frontal(L)-Occipital(L)#theta', 'COH#Frontal(L)-Occipital(R)#alpha1', 'COH#Frontal(R)-Occipital(L)#alpha1', 'COH#Parietal(R)-Occipital(L)#alpha1', 'COH#Frontal(L)-Frontal(R)#alpha2', 'COH#Frontal(L)-Occipital(R)#alpha2', 'COH#Parietal(R)-Occipital(L)#alpha2', 'COH#Parietal(R)-Occipital(R)#alpha2', 'COH#Occipital(L)-Occipital(R)#alpha2', 'COH#Frontal(L)-Occipital(L)#beta1', 'COH#Temporal(R)-Parietal(R)#beta1', 'COH#Parietal(R)-Occipital(R)#beta1', 'COH#Frontal(L)-Parietal(L)#beta2', 'COH#Frontal(R)-Occipital(L)#beta2', 'COH#Frontal(L)-Temporal(R)#beta3', 'COH#Frontal(L)-Parietal(L)#beta3', 'COH#Frontal(L)-Occipital(L)#beta3', 'COH#Frontal(L)-Occipital(R)#beta3', 'COH#Frontal(R)-Occipital(L)#beta3', 'COH#Temporal(L)-Occipital(R)#beta3', 'COH#Frontal(L)-Occipital(R)#gamma', 'COH#Frontal(R)-Occipital(R)#gamma']


In [6]:
underrepresented_targets = [4, 6, 9]

In [7]:
# Read all targets
all_targets = pd.read_csv('/Volumes/MMIS-Saraiv/Datasets/Miltiadous Dataset/participants.tsv', index_col=0, sep='\t')
# Remove "sub-" from the index and make them integers
all_targets.index = all_targets.index.str.replace('sub-', '')
all_targets.index = all_targets.index.astype(int)
all_targets

In [8]:
# Read all and standardize
all_features = []
for subject_path in all_directories:
    subject = subject_path.split('/')[-1]
        
    # Get feature files
    all_files = glob(join(subject_path, '*.csv'))
    # remove all files that have "Multiple" in their name
    all_files = [file for file in all_files if 'Multiple' not in file]
    # Read them all
    all_data = [pd.read_csv(file, index_col=0) for file in all_files]
    # FIX: the spectral data has a different index
    for data in all_data:
        if 'Spectral' in data.columns[0]:
            data.index = [int(subject),]
    # Concatenate all their columns in a single DataFrame
    all_data = pd.concat(all_data, axis=1)
    # Keep only selected features
    selected_features = all_data[FEATURES_SELECTED]
    # drop nans
    selected_features = selected_features.dropna()
    
    all_features.append(selected_features)
    
all_features = pd.concat(all_features, axis=0)

# Standardize feature-wise
all_features = (all_features - all_features.mean()) / all_features.std()
all_features

In [9]:
# Append target column
all_features['target'] = all_targets['MMSE']
all_features

In [14]:
# Apply SMOTE-R to balance the dataset
import smogn

# Apply SMOGN to balance the dataset
balanced_data = smogn.smoter(
    data = all_features, 
    y = 'target'
)
balanced_data_targets = balanced_data['target']
balanced_data_targets


In [20]:
# Plot target distribution before and after
import matplotlib.pyplot as plt

plt.hist(all_features['target'], bins=30)
plt.title("Before")
plt.ylim((0, 30))
plt.show()

plt.hist(balanced_data_targets, bins=30)
plt.title("After")
plt.ylim((0, 30))
plt.show()